In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

데이터 불러오기 

data1 : phenotype_1000

data2 : KCPS

data3 : phenotype_1000 + KCPS

In [2]:
data1=pd.read_csv('C:/Users/dudtj/OneDrive - 숭실대학교 - Soongsil University/Desktop/CL/python/동계인턴십_Data_1000/phenotype_1000.txt',engine="python",sep=" ")

In [3]:
data2=pd.read_csv('C:/Users/dudtj/OneDrive - 숭실대학교 - Soongsil University/Desktop/CL/python/동계인턴십_Data_1000/KCPS_r2_0.01_1000.csv')

In [4]:
for i in range(2,15190):
    data2.columns.values[i]=i

In [5]:
data1=data1.loc[:,['LUNG','FID','AGE_B', 'SMOK_B', 'ALCO_B','EXER_B','HT_B', 'WT_B', 'WAIST_B', 'SBP_B', 'DBP_B', 'CHO_B','LDL_B', 'TG_B', 'HDL_B', 'FBS_B', 'GOT_B', 'GPT_B', 'GGT_B', 'URIC_B','BIL', 'WBC', 'CREAT','SEX1', 'CRC']]

In [6]:
import matplotlib.pyplot as plt 
import seaborn as sns

In [7]:
#corr = data1.corr(method='pearson')
#colormap = plt.cm.PuBu 
#plt.figure(figsize=(15,15)) 
#plt.title("Person Correlation of Features", y = 1.05, size = 15) 
#sns.heatmap(data1.astype(float).corr(),fmt='.2f', linewidths = 0.1, vmax = 1.0, square = True, cmap = colormap, linecolor = "white", annot = True, annot_kws = {"size" : 10})

In [8]:
data3 = pd.concat([data1,data2],axis=1)

In [9]:
data1.dropna(inplace=True)
data1.drop('FID',axis=1,inplace=True)

In [10]:
data1

,LUNG,AGE_B,SMOK_B,ALCO_B,EXER_B,HT_B,WT_B,WAIST_B,SBP_B,DBP_B,...,FBS_B,GOT_B,GPT_B,GGT_B,URIC_B,BIL,WBC,CREAT,SEX1,CRC
0,0,48,3.0,2.0,1.0,168.0,69.8,82.0,102.0,69.0,...,79.0,18.0,14.0,20.0,5.0,1.6,9.57,1.1,1,0
1,0,53,1.0,2.0,1.0,161.0,67.9,86.0,127.0,67.0,...,126.0,15.0,13.0,15.0,3.7,0.8,8.80,0.8,2,0
2,0,60,1.0,1.0,2.0,153.0,73.3,97.0,133.0,76.0,...,157.0,56.0,88.0,41.0,6.1,0.8,7.19,0.8,2,0
3,0,42,1.0,2.0,2.0,167.0,65.1,77.5,97.0,70.0,...,89.0,17.0,14.0,12.0,3.2,0.4,4.65,0.9,1,0
4,0,57,3.0,2.0,2.0,165.0,51.8,73.0,98.0,76.0,...,120.0,13.0,11.0,14.0,5.0,0.4,6.56,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,0,37,3.0,2.0,2.0,173.0,69.4,82.0,145.0,87.0,...,83.0,27.0,19.0,42.0,3.4,0.4,6.10,1.0,1,0
996,0,57,2.0,2.0,1.0,159.0,61.4,87.0,109.0,80.0,...,120.0,24.0,23.0,34.0,3.7,0.9,7.60,1.0,1,0
997,0,50,2.0,2.0,1.0,163.0,66.4,82.0,150.0,82.0,...,87.0,19.0,13.0,46.0,4.4,0.6,5.80,0.7,2,0
998,0,45,1.0,2.0,1.0,156.0,48.0,67.5,91.0,60.0,...,82.0,30.0,24.0,14.0,2.1,0.6,7.20,0.7,2,0


In [11]:
#data1.drop('LUNG',axis=1,inplace=True)
#X_df = data1

In [12]:
y_df = data1.iloc[:, 0]

In [13]:
data1.drop('LUNG',axis=1,inplace=True)
X_df = data1

In [14]:
X_df

,AGE_B,SMOK_B,ALCO_B,EXER_B,HT_B,WT_B,WAIST_B,SBP_B,DBP_B,CHO_B,...,FBS_B,GOT_B,GPT_B,GGT_B,URIC_B,BIL,WBC,CREAT,SEX1,CRC
0,48,3.0,2.0,1.0,168.0,69.8,82.0,102.0,69.0,196.0,...,79.0,18.0,14.0,20.0,5.0,1.6,9.57,1.1,1,0
1,53,1.0,2.0,1.0,161.0,67.9,86.0,127.0,67.0,159.0,...,126.0,15.0,13.0,15.0,3.7,0.8,8.80,0.8,2,0
2,60,1.0,1.0,2.0,153.0,73.3,97.0,133.0,76.0,198.0,...,157.0,56.0,88.0,41.0,6.1,0.8,7.19,0.8,2,0
3,42,1.0,2.0,2.0,167.0,65.1,77.5,97.0,70.0,139.0,...,89.0,17.0,14.0,12.0,3.2,0.4,4.65,0.9,1,0
4,57,3.0,2.0,2.0,165.0,51.8,73.0,98.0,76.0,166.0,...,120.0,13.0,11.0,14.0,5.0,0.4,6.56,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,37,3.0,2.0,2.0,173.0,69.4,82.0,145.0,87.0,156.0,...,83.0,27.0,19.0,42.0,3.4,0.4,6.10,1.0,1,0
996,57,2.0,2.0,1.0,159.0,61.4,87.0,109.0,80.0,194.0,...,120.0,24.0,23.0,34.0,3.7,0.9,7.60,1.0,1,0
997,50,2.0,2.0,1.0,163.0,66.4,82.0,150.0,82.0,210.0,...,87.0,19.0,13.0,46.0,4.4,0.6,5.80,0.7,2,0
998,45,1.0,2.0,1.0,156.0,48.0,67.5,91.0,60.0,159.0,...,82.0,30.0,24.0,14.0,2.1,0.6,7.20,0.7,2,0


In [15]:
y_df

0      0
1      0
2      0
3      0
4      0
      ..
991    0
996    0
997    0
998    0
999    0
Name: LUNG, Length: 788, dtype: int64

In [16]:
#y_df =data1['LUNG']
#X_df =data1.drop('LUNG', axis=1)

In [17]:
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score # 평가를 위함
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler, Binarizer

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [18]:
# random_state = 156 : 다음에도 같은 데이터를 train, test로 분리 하는 옵션
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_df,y_df,test_size=0.5,random_state=156,stratify=y_df)

In [19]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred) # 오차 행렬 (평가 지표)
    accuracy = accuracy_score(y_test, pred) # 정확도
    precision = precision_score(y_test, pred) # 정밀도
    recall = recall_score(y_test, pred) # 재현율
    f1 = f1_score(y_test, pred) # F1 (정밀도, 재현율 적정 수준 : 1에 가까울 수록 좋음)
    roc_auc = roc_auc_score(y_test, pred_proba) # AUC (1에 가까울 수록 좋음)
    
    print('오차 행렬\n {}'.format(confusion))
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, F1 : {3:.4f}, AUC: {4:.4f}'
         .format(accuracy, precision, recall, f1, roc_auc))

In [20]:
def precision_recall_curve_plot(y_test=None, pred_proba_c1=None):
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출. 
    precisions, recalls, thresholds = precision_recall_curve( y_test, pred_proba_c1)
    
    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary],label='recall')
    
    # threshold 값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    
    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

In [21]:
#random forest
random = RandomForestClassifier()
random.fit(X_train, y_train)
pred = random.predict(X_test)
pred_proba = random.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[383   0]
 [ 11   0]]
정확도 : 0.9721, 정밀도 : 0.0000, 재현율 : 0.0000, F1 : 0.0000, AUC: 0.5576


In [22]:
#tree
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
pred_proba = tree.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[366  17]
 [ 11   0]]
정확도 : 0.9289, 정밀도 : 0.0000, 재현율 : 0.0000, F1 : 0.0000, AUC: 0.4778


In [23]:
#adaboost
adaboost = AdaBoostClassifier()
adaboost.fit(X_train, y_train)
pred = adaboost.predict(X_test)
pred_proba = adaboost.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[373  10]
 [ 11   0]]
정확도 : 0.9467, 정밀도 : 0.0000, 재현율 : 0.0000, F1 : 0.0000, AUC: 0.6537


In [24]:
#xgboost
xgboost = XGBClassifier()
xgboost.fit(X_train, y_train)
pred = xgboost.predict(X_test)
pred_proba = xgboost.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

[17:33:15] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
오차 행렬
 [[382   1]
 [ 11   0]]
정확도 : 0.9695, 정밀도 : 0.0000, 재현율 : 0.0000, F1 : 0.0000, AUC: 0.4617


In [25]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators=400)
lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
pred_proba = lgbm.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)


오차 행렬
 [[382   1]
 [ 11   0]]
정확도 : 0.9695, 정밀도 : 0.0000, 재현율 : 0.0000, F1 : 0.0000, AUC: 0.5151
